In [ ]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px
import json
pd.set_option('display.max_colwidth', None)
import plotly.express as px
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import plotly.graph_objs as go
import numpy as np

# from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter

from transformers import AutoTokenizer
from datasets import Dataset
from transformers import BertForTokenClassification, Trainer, TrainingArguments


import os

from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
file_path = '/content/drive/MyDrive/compliance/ner/'

In [ ]:
train_df = pd.read_csv(file_path + 'gdpr-compliant-ner.conllgdpr_train.csv')
val_df = pd.read_csv(file_path + 'gdpr-compliant-ner.conllgdpr_val.csv')
test_df = pd.read_csv(file_path + 'gdpr-compliant-ner.conllgdpr_test.csv')

print(train_df.shape, val_df.shape, test_df.shape)

(189999, 3) (40715, 3) (40716, 3)


In [ ]:
train_df.head()

,Word,Tag,tag_group
0,purposes,I-NRP,NRP
1,target,I-LI,LI
2,above,O,O
3,or,O,O
4,redirect,O,O


In [ ]:
unique_labels = train_df['Tag'].unique()
unique_labels_val = val_df['Tag'].unique()
unique_labels_test = test_df['Tag'].unique()
# Display the unique labels
print(f"Unique labels in the training set: {unique_labels}")
print(f"Total number of unique labels: {len(unique_labels)}")

print(f"Unique labels in the training set: {unique_labels_val}")
print(f"Total number of unique labels: {len(unique_labels_val)}")


print(f"Unique labels in the training set: {unique_labels_test}")
print(f"Total number of unique labels: {len(unique_labels_test)}")


Unique labels in the training set: ['I-NRP' 'I-LI' 'O' 'I-PD' 'I-TM' 'I-RP' 'I-DSR20' 'B-PD' 'I-P' 'I-RET'
 'I-R' 'I-CONS' 'I-OM' 'B-P' 'B-DC' 'I-NPD' 'I-DSO' 'I-LB' 'B-CONS' 'B-DS'
 'B-TP' 'I-DSR21' 'I-TP' 'I-DSR17' 'I-RI' 'I-CONT' 'I-DSR15' 'B-RP'
 'I-SNEU' 'I-A' 'B-LB' 'B-NRP' 'B-CONT' 'B-SNEU' 'I-DC' 'B-OM' 'B-LI'
 'B-DSO' 'B-NPD' 'I-DP' 'B-SEU' 'B-LC' 'B-DPO' 'B-RI' 'I-DPO' 'I-DS'
 'B-TM' 'B-R' 'B-DP' 'B-DSR21' 'I-ADM' 'I-SEU' 'B-ADM' 'B-DSR17' 'I-DSR18'
 'I-DSR16' 'B-RET' 'B-A' 'I-LC' 'B-DSR15' 'B-DSR16' 'B-DSR18' 'I-DSR19'
 'I-DSR22' 'B-DSR20' 'B-DSR19' 'B-DSR22']
Total number of unique labels: 67
Unique labels in the training set: ['O' 'I-DSO' 'I-TM' 'B-DC' 'I-P' 'I-RP' 'I-CONS' 'I-PD' 'I-RET' 'I-NRP'
 'I-CONT' 'I-NPD' 'I-OM' 'I-R' 'B-LB' 'I-DSR16' 'B-RP' 'I-RI' 'I-LI'
 'I-TP' 'I-LB' 'I-LC' 'B-PD' 'I-DC' 'B-P' 'I-SNEU' 'B-CONS' 'B-R'
 'I-DSR17' 'I-DSR15' 'B-DS' 'I-DP' 'B-DSO' 'B-TP' 'B-RI' 'B-TM' 'B-RET'
 'I-DSR20' 'I-DS' 'I-DSR21' 'B-SEU' 'B-SNEU' 'B-NPD' 'B-LI' 'B-OM' 'B-NRP'

### 2. Load tokenizer

In [ ]:
def create_label_to_id(df):
    """
    Create a dynamic label-to-ID mapping from the NER tags in the dataframe.

    Args:
        df (DataFrame): Dataframe with 'Tag' column containing NER labels.

    Returns:
        label_to_id (dict): A dictionary mapping each unique label to an integer ID.
    """
    unique_labels = df['Tag'].unique()
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    return label_to_id

label_to_id = create_label_to_id(train_df)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("nlpaueb/legal-bert-base-uncased", num_labels=len(label_to_id))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_and_align_labels(df, tokenizer, label_to_id, max_length=512, chunk_size=100):
    """
    Tokenizes words from the dataframe and aligns the NER labels with tokenized outputs by grouping words into chunks.
    """
    input_ids_list = []
    attention_mask_list = []
    aligned_labels_list = []

    words_list = df['Word'].tolist()
    labels_list = df['Tag'].tolist()


    for i in range(0, len(words_list), chunk_size):
        chunk_words = words_list[i:i + chunk_size]
        chunk_labels = labels_list[i:i + chunk_size]

        tokenized_inputs = tokenizer(
            chunk_words,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            is_split_into_words=True,
            return_tensors='pt'
        )

        word_ids = tokenized_inputs.word_ids()
        aligned_labels = []

        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(label_to_id.get(chunk_labels[word_idx], -100))
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx

        input_ids_list.append(tokenized_inputs['input_ids'][0].tolist())
        attention_mask_list.append(tokenized_inputs['attention_mask'][0].tolist())
        aligned_labels_list.append(aligned_labels)

    return {
        'input_ids': input_ids_list,
        'attention_mask': attention_mask_list,
        'labels': aligned_labels_list
    }


In [ ]:
sample_text = ["This", "is", "a", "test"]
tokenized_sample = tokenizer(
    sample_text,
    truncation=True,
    padding=True,
    max_length=512,
    is_split_into_words=True
)

print(tokenized_sample['input_ids'])


[101, 226, 223, 145, 893, 102]


In [ ]:
train_encodings = tokenize_and_align_labels(train_df, tokenizer, label_to_id)
val_encodings = tokenize_and_align_labels(val_df, tokenizer, label_to_id)
test_encodings = tokenize_and_align_labels(test_df, tokenizer, label_to_id)


In [ ]:
print(train_encodings.keys())
print(train_encodings['input_ids'][:5])
print(train_encodings['labels'][:5])


dict_keys(['input_ids', 'attention_mask', 'labels'])
[[101, 504, 1356, 348, 215, 21189, 117, 7647, 252, 117, 115, 117, 1216, 323, 117, 145, 212, 532, 1590, 223, 794, 117, 117, 5102, 479, 231, 215, 799, 843, 174, 4444, 2107, 14098, 231, 2972, 1260, 410, 215, 2551, 215, 685, 223, 1216, 115, 1038, 808, 353, 11553, 440, 237, 115, 225, 808, 459, 3423, 355, 10252, 211, 115, 2796, 6853, 211, 115, 2743, 338, 112, 532, 245, 117, 223, 221, 11957, 369, 117, 5004, 6805, 3703, 189, 578, 115, 668, 18462, 21232, 5316, 644, 238, 207, 365, 283, 537, 2126, 1363, 226, 222, 1030, 222, 372, 799, 225, 228, 237, 532, 242, 1410, 245, 117, 276, 808, 117, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)
test_dataset = Dataset.from_dict(test_encodings)


In [ ]:
print(train_encodings.keys())
print(type(train_encodings['input_ids'][0]))

dict_keys(['input_ids', 'attention_mask', 'labels'])
<class 'list'>


In [ ]:
# Load pre-trained model
# NUM_CLASSES = 67
# model = BertForTokenClassification.from_pretrained(
#     "dbmdz/bert-large-cased-finetuned-conll03-english",
#     num_labels=NUM_CLASSES,
#     ignore_mismatched_sizes=True
# )

output_dir = '/content/drive/MyDrive/compliance/ner/NER_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.545900,1.527156
2,1.462000,1.446639
3,1.438300,1.433555


TrainOutput(global_step=357, training_loss=1.5389001162446179, metrics={'train_runtime': 581.9062, 'train_samples_per_second': 9.795, 'train_steps_per_second': 0.614, 'total_flos': 1490266770739200.0, 'train_loss': 1.5389001162446179, 'epoch': 3.0})

In [ ]:
# Assuming you have a Trainer object
eval_result = trainer.evaluate()

# Print evaluation results
print(eval_result)


{'eval_loss': 1.4335551261901855, 'eval_runtime': 12.8669, 'eval_samples_per_second': 31.709, 'eval_steps_per_second': 2.021, 'epoch': 3.0}


In [ ]:
# Assuming `val_dataset` is your validation dataset
predictions = trainer.predict(val_dataset)

# Extract the predictions
preds = predictions.predictions

# Print or analyze the predictions
print(preds)


[[[-2.1234325e-01  6.2917703e-01  4.4787931e+00 ... -1.3872931e+00
   -1.5999799e+00 -2.3031218e+00]
  [-5.4524362e-01  5.5780774e-01  4.8759079e+00 ... -2.3043475e+00
   -2.2015595e+00 -1.7160481e+00]
  [ 2.2603866e-02  2.4221110e+00  5.8287144e+00 ... -2.0311375e+00
   -1.9520576e+00 -2.1657441e+00]
  ...
  [-6.6700453e-01  3.9880061e-01  5.7844849e+00 ... -1.4701127e+00
   -1.9014935e+00 -1.7074798e+00]
  [-6.2570715e-01  5.8109307e-01  5.5236492e+00 ... -1.5646727e+00
   -1.8518691e+00 -1.6783319e+00]
  [-2.6838315e-01  3.9558977e-01  1.9856720e+00 ... -8.5873973e-01
   -5.0943404e-01 -6.9526982e-01]]

 [[-1.8844834e-01  5.9113085e-01  4.6582470e+00 ... -1.3464617e+00
   -1.5880132e+00 -2.3748481e+00]
  [-4.8319191e-02  1.8116806e+00  6.2390490e+00 ... -2.0683208e+00
   -1.9427326e+00 -2.4343977e+00]
  [-6.1087860e-03  2.2043631e+00  5.9323750e+00 ... -2.0552924e+00
   -2.1241796e+00 -2.2346308e+00]
  ...
  [-5.7647347e-01  3.9980191e-01  5.8083353e+00 ... -1.4451197e+00
   -1.9438

In [ ]:
import numpy as np

predicted_labels = np.argmax(preds, axis=2)

# Print predicted labels for the validation set
print(predicted_labels)


[[ 2  2  2 ...  2  2  2]
 [ 2  2  2 ...  2  2  2]
 [ 2  2  2 ...  2  2  2]
 ...
 [ 2  2  2 ...  2  2  2]
 [ 2  2  2 ...  2  2  2]
 [ 2  2  2 ...  2  2 52]]


In [ ]:
# Assuming you already have `label_to_id` from before
id_to_label = {v: k for k, v in label_to_id.items()}

# Now you can use this `id_to_label` to map the numerical predictions back to their original tags


In [ ]:
true_labels = val_encodings['labels']
print(true_labels)

[[-100, 2, 2, 16, 2, 2, 4, 14, 2, 8, -100, 5, 2, 2, 2, -100, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 2, 11, 2, 2, 2, 16, 2, 2, 2, 2, 3, 9, 16, 2, 5, 2, 2, 0, 2, 2, 14, 2, 2, 25, 2, 2, 2, 2, 5, -100, 2, 2, 2, 2, 2, 5, 2, 5, 15, 14, -100, 12, 10, 2, -100, -100, 3, 2, 2, 2, 2, 2, -100, 2, 2, 30, 55, 27, 2, 2, -100, -100, 2, 15, 2, 2, 24, 2, 12, 2, 12, 2, 1, 2, 5, 2, 2, 2, 2, 2, 12, 2, 3, 2, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -

In [ ]:
filtered_true_tags = []
filtered_predicted_tags = []
reshaped_true_labels = np.array(true_labels).reshape(predicted_labels.shape)

for true_seq, pred_seq in zip(reshaped_true_labels, predicted_labels):
    filtered_true_seq = []
    filtered_pred_seq = []

    for true_label, pred_label in zip(true_seq, pred_seq):
        if true_label != -100:
            filtered_true_seq.append(id_to_label[true_label])
            filtered_pred_seq.append(id_to_label[pred_label])

    filtered_true_tags.extend(filtered_true_seq)
    filtered_predicted_tags.extend(filtered_pred_seq)

from sklearn.metrics import classification_report
report = classification_report(filtered_true_tags, filtered_predicted_tags)
print(report)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

         B-A       0.00      0.00      0.00        17
       B-ADM       0.67      0.21      0.32        19
      B-CONS       0.30      0.25      0.27       110
      B-CONT       0.00      0.00      0.00        24
        B-DC       0.38      0.07      0.12       253
        B-DP       0.00      0.00      0.00        25
       B-DPO       0.00      0.00      0.00        10
        B-DS       0.00      0.00      0.00        87
       B-DSO       0.00      0.00      0.00        60
     B-DSR15       0.00      0.00      0.00        11
     B-DSR16       0.00      0.00      0.00         7
     B-DSR17       0.00      0.00      0.00        14
     B-DSR18       0.00      0.00      0.00         8
     B-DSR19       0.00      0.00      0.00         1
     B-DSR20       0.00      0.00      0.00         5
     B-DSR21       0.00      0.00      0.00         8
     B-DSR22       0.00      0.00      0.00         1
        B-LB       0.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

true_labels = predictions.label_ids.flatten()
flat_predictions = np.argmax(predictions.predictions, axis=2).flatten()

accuracy = accuracy_score(true_labels, flat_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, flat_predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Accuracy: 0.12923177083333334
Precision: 0.034162615829092864
Recall: 0.12923177083333334
F1-Score: 0.0359570954172553


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation

In [ ]:
import pandas as pd
import torch

def evaluate_on_unlabeled_text_to_dataframe(policy_text, tokenizer, model, id_to_label, max_length=512):
    tokenized_inputs = tokenizer(
        policy_text.split(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        is_split_into_words=True,
        return_tensors='pt'
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

    with torch.no_grad():
        output = model(**tokenized_inputs)

    predicted_ids = torch.argmax(output.logits, dim=2).squeeze().tolist()

    predicted_tags = [id_to_label[pred_id] for pred_id in predicted_ids]

    tokenized_words = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'].squeeze().tolist())

    df = pd.DataFrame(list(zip(tokenized_words, predicted_tags)), columns=['Word', 'Tag'])

    df = df[df['Word'] != '[PAD]']
    df = df[~df['Word'].str.startswith('##')]

    return df

policy_text = "When you are purchasing premium services or products through the Curse Features you may have to submit personal information to our billing partner, PayPal, including your email address, billing address and additional billing information depending on your preferred method of payment through PayPal, such as your credit card number. When you enter a contest or sweepstakes with Curse, in addition to your screen name, name and email address, we will ask for your mailing address in case we have to mail you a prize, and may ask you to opt into receiving a third party's offers and/or newsletter."

predictions_df = evaluate_on_unlabeled_text_to_dataframe(policy_text, tokenizer, model, id_to_label)

predictions_df[predictions_df['Tag'] != 'O']

,Word,Tag
21,personal,B-PD
33,email,B-PD
34,address,I-PD
37,address,I-PD
56,credit,B-PD
57,card,I-PD
58,number,I-PD
79,name,I-PD
81,name,I-PD
83,email,B-PD


With two policy segments

In [ ]:
import pandas as pd
import torch

def evaluate_on_unlabeled_text_to_dataframe(policy_text, tokenizer, model, id_to_label, max_length=512):
    tokenized_inputs = tokenizer(
        policy_text.split(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        is_split_into_words=True,
        return_tensors='pt'
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    tokenized_inputs = {key: value.to(device) for key, value in tokenized_inputs.items()}

    with torch.no_grad():
        output = model(**tokenized_inputs)

    predicted_ids = torch.argmax(output.logits, dim=2).squeeze().tolist()

    predicted_tags = [id_to_label[pred_id] for pred_id in predicted_ids]

    tokenized_words = tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'].squeeze().tolist())

    df = pd.DataFrame(list(zip(tokenized_words, predicted_tags)), columns=['Word', 'Tag'])

    df = df[df['Word'] != '[PAD]']
    df = df[~df['Word'].str.startswith('##')]

    return df

policy_text = """
When you are purchasing premium services or products through the Curse Features you may have to submit personal information
to our billing partner, PayPal, including your email address, billing address and additional billing information depending on
your preferred method of payment through PayPal, such as your credit card number. When you enter a contest or sweepstakes
with Curse, in addition to your screen name, name and email address, we will ask for your mailing address in case we have to
mail you a prize, and may ask you to opt into receiving a third party's offers and/or newsletter.

This Privacy Policy does not cover the information practices of third party websites linked to the Site(s) or third party offer
and/or newsletter opt-ins provided through the Site(s). Although we encourage such third parties to adopt and follow their own
privacy policies, we are not responsible for their collection and use of your personal information. You should refer to the
privacy policies and statements of other websites or contact the respective webmasters of those websites to obtain information
regarding their information collection, use and disclosure policies.
"""

predictions_df = evaluate_on_unlabeled_text_to_dataframe(policy_text, tokenizer, model, id_to_label)

predictions_df = predictions_df[predictions_df['Tag'] != 'O']

print(predictions_df)


           Word     Tag
21     personal    B-PD
33        email    B-PD
34      address    I-PD
37      address    I-PD
56       credit    B-PD
57         card    I-PD
58       number    I-PD
64      contest    I-RP
79         name    I-PD
81         name    I-PD
83        email    B-PD
84      address    I-PD
92      address    I-PD
112       third    B-TP
133       party    I-TP
145       party    I-TP
151         opt  B-CONS
168     parties    I-TP
184  collection     B-P
189    personal    B-PD
